In [1]:
import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

## データセットの読み込み

iris の分類は 4 次元の特徴量を使用した 3 値分類のタスク


In [2]:
# データセットの読み込み
iris = sns.load_dataset("iris")
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
# カテゴリ変数を数値に変換
species_map = {"setosa": 0, "versicolor": 1, "virginica": 2}
iris["species"] = iris["species"].map(species_map)
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


PyTorch を使用する場合は、torch.Tensor 型に変換する必要があるので、torch.Tensor 型に変換。

ここで注意すべきことは、ターゲット t の型であり、これをタスクによって変える必要がある。(適切な損失関数がタスクによって異なるため。以下()内には代表的な損失関数を記載)

- 回帰の場合: torch.float32(MSE など)
- 二値分類の場合: torch.float32(Binary Cross Entropy; BCE など)
- 多値分類の場合: torch.int64(Cross Entropy など)

今回は 3 値分類のため、torch.int64 とする。


In [4]:
class IrisDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        features: list[str],
        labels: list[str],
    ):
        self.features = df[features].values
        self.labels = df[labels].values.astype(
            np.int64
        )  # 今回は3値分類のためtarget変数はint64

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.int64)
        return feature, label

In [5]:
# データセットの作成
iris_dataset = IrisDataset(
    iris, ["sepal_length", "sepal_width", "petal_length", "petal_width"], ["species"]
)
print(len(iris_dataset))
print(iris_dataset[100])

150


(tensor([6.3000, 3.3000, 6.0000, 2.5000]), tensor([2]))


In [6]:
# Datasetの分割
## train:val:test = 8:1:1
n_train = int(len(iris_dataset) * 0.8)
n_val = int(len(iris_dataset) * 0.1)
n_test = int(len(iris_dataset) * 0.1)

torch.manual_seed(123)
train, val, test = torch.utils.data.random_split(iris_dataset, [n_train, n_val, n_test])

ミニバッチ学習を実施するために Data Loder を用意。

- batch_size：ミニバッチのサイズ
- shuffle：ミニバッチを作成する際にデータをシャッフルするか
- drop_last：バッチサイズがデータサンプルに対して割り切れない場合に除外するか


In [7]:
# バッチサイズの定義
# batch_size = 10

# Data Loader を用意
## shuffle はデフォルトで False のため、訓練データのみ True に指定
train_loader = torch.utils.data.DataLoader(
    train, batch_size=10, shuffle=True, drop_last=True
)
val_loader = torch.utils.data.DataLoader(val, batch_size=10)
test_loader = torch.utils.data.DataLoader(test, batch_size=10)

## ネットワークの定義


[pytorch.ipynb](Python/pytorch/pytorch.ipynb)で記載した Pytorch そのままの学習〜推論の処理と比較しながら処理を確認すると pytorch lightning の処理の簡潔さが分かりやすい。


順伝搬は、「全結合 → 活性化(ReLU)→ 全結合 →Softmax」とする。
※このとき、forward 関数に Softmax 処理の記述がないのは、損失関数である cross_entropy 内に softmax の処理も入っているため。


In [8]:
class IrisNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(4, 10),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(10, 10),
            nn.ReLU(),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(10, 3),
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        out = self.layer3(x)
        return out

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        y_label = torch.argmax(y, dim=1)
        loss = F.cross_entropy(y, t.squeeze_())
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        y_label = torch.argmax(y, dim=1)
        loss = F.cross_entropy(y, t.squeeze_())
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        y_label = torch.argmax(y, dim=1)
        loss = F.cross_entropy(y, t.squeeze_())
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        self.log("test_loss", loss, on_step=False, on_epoch=True)
        self.log("test_acc", acc, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.1)
        return optimizer

In [9]:
# シードを固定して再現性を確保
pl.seed_everything(0)

# 学習を行う Trainer
net = IrisNet()
trainer = pl.Trainer(max_epochs=30, deterministic=True)

Seed set to 0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
# 学習の実行
trainer.fit(net, train_loader, val_loader)


  | Name   | Type       | Params
--------------------------------------
0 | layer1 | Sequential | 50    
1 | layer2 | Sequential | 110   
2 | layer3 | Sequential | 33    
--------------------------------------
193       Trainable params
0         Non-trainable params
193       Total params
0.001     Total estimated model params size (MB)


Sanity Checking: |                                                                                            …

/opt/conda/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=30` reached.


In [13]:
# テストデータで検証
trainer.validate(dataloaders=val_loader)
trainer.test(dataloaders=test_loader)
# 結果表示
trainer.callback_metrics

Restoring states from the checkpoint path at /home/chinchilla/TIL/Python/machine_learning/pytorch/lightning_logs/version_0/checkpoints/epoch=29-step=360.ckpt
Loaded model weights from the checkpoint at /home/chinchilla/TIL/Python/machine_learning/pytorch/lightning_logs/version_0/checkpoints/epoch=29-step=360.ckpt


Validation: |                                                                                                 …

Restoring states from the checkpoint path at /home/chinchilla/TIL/Python/machine_learning/pytorch/lightning_logs/version_0/checkpoints/epoch=29-step=360.ckpt
Loaded model weights from the checkpoint at /home/chinchilla/TIL/Python/machine_learning/pytorch/lightning_logs/version_0/checkpoints/epoch=29-step=360.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc                    1.0
        val_loss            0.18407855927944183
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Testing: |                                                                                                    …

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        test_loss           0.2226889282464981
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


{'test_loss': tensor(0.2227), 'test_acc': tensor(1.)}